In [5]:
#!pip3 install tensorflow
#!pip3 install tensorflow-datasets

In [6]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Tensorflow libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Input
from tensorflow.keras.layers import (
    GRU,
    Add,
    AdditiveAttention,
    Attention,
    Concatenate,
    Dense, 
    Embedding,
    LayerNormalization,
    Reshape,
    StringLookup,
    TextVectorization
)









/Users/kavisanthoshkumar/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
